## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-12-13-00-50 +0000,nypost,Gov. Hochul asks Trump for power to disable ne...,https://nypost.com/2025/07/12/us-news/state-se...
1,2025-07-12-12-58-56 +0000,nypost,Soros funneled $37 million to Working Families...,https://nypost.com/2025/07/12/us-news/soros-fu...
2,2025-07-12-12-57-07 +0000,nyt,Air India Plane Crash Report Says Fuel to Engi...,https://www.nytimes.com/2025/07/11/world/asia/...
3,2025-07-12-12-54-06 +0000,nypost,Do you live in the most perverted state in Ame...,https://nypost.com/2025/07/12/us-news/do-you-l...
4,2025-07-12-12-53-00 +0000,nypost,Beautiful inside and out: Air Force vet who do...,https://nypost.com/2025/07/12/us-news/air-forc...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2310/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
3,trump,7
257,canada,4
18,mamdani,3
36,out,3
58,nyc,3


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
8,2025-07-12-12-20-00 +0000,nypost,Accused Minnesota assassin Vance Boelter hints...,https://nypost.com/2025/07/12/world-news/accus...,26
40,2025-07-11-14-53-00 +0000,wsj,Businesses hoping for clarity on where Trump t...,https://www.wsj.com/economy/trade/trump-tariff...,25
30,2025-07-11-19-40-14 +0000,nyt,Trump’s Latest Canada Tariff Threats Come Desp...,https://www.nytimes.com/2025/07/11/world/canad...,21
25,2025-07-12-01-53-00 +0000,wsj,A dispute over the Federal Reserve’s renovatio...,https://www.wsj.com/economy/central-banking/je...,19
13,2025-07-12-09-57-58 +0000,nypost,Arizona homeowner David Martin fined $100 repe...,https://nypost.com/2025/07/12/us-news/arizona-...,18


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
8,26,2025-07-12-12-20-00 +0000,nypost,Accused Minnesota assassin Vance Boelter hints...,https://nypost.com/2025/07/12/world-news/accus...
25,19,2025-07-12-01-53-00 +0000,wsj,A dispute over the Federal Reserve’s renovatio...,https://www.wsj.com/economy/central-banking/je...
13,18,2025-07-12-09-57-58 +0000,nypost,Arizona homeowner David Martin fined $100 repe...,https://nypost.com/2025/07/12/us-news/arizona-...
40,18,2025-07-11-14-53-00 +0000,wsj,Businesses hoping for clarity on where Trump t...,https://www.wsj.com/economy/trade/trump-tariff...
19,16,2025-07-12-04-50-49 +0000,nypost,"Alleged Florida child rapist lured girl, 11, i...",https://nypost.com/2025/07/12/us-news/alleged-...
35,16,2025-07-11-17-37-00 +0000,wsj,Nuclear power exemplifies how revamping dated ...,https://www.wsj.com/economy/trumps-unsung-econ...
9,16,2025-07-12-12-15-00 +0000,nypost,NYC educator put in for bereavement leave 21 ...,https://nypost.com/2025/07/12/us-news/nyc-educ...
24,15,2025-07-12-02-41-31 +0000,nypost,Texas family mourns ‘shining light’ 8-year-old...,https://nypost.com/2025/07/11/us-news/texas-fa...
1,14,2025-07-12-12-58-56 +0000,nypost,Soros funneled $37 million to Working Families...,https://nypost.com/2025/07/12/us-news/soros-fu...
18,14,2025-07-12-04-51-59 +0000,nypost,Beloved ‘Wheelchair Waver’ critically injured ...,https://nypost.com/2025/07/12/us-news/wheelcha...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
